Открываем файл:

In [154]:
import pandas as pd
import numpy as np

df = pd.read_csv("mtesrl_20150626_MD0000600012_stats.txt", header = 1, skipfooter = 2, sep = "\t", usecols = ['EVENT', 'AVGTSMR'], engine='python')

## Часть 1.

Находим информацию по каждому типу транзакций (EVENT).  
  
- EVENTNAME - Название транзакции  
- min - Минимальное время ответа на транзакцию в микросекундах  
- 50% - Медиана  
- 90% - 90% результатов меньше 122 микросекунд  
- 99% - 99% результатов меньше 140 микросекунд  
- 99.9% - 99.9% результатов меньше 145 микросекунд  

In [155]:
stat = df.groupby('EVENT').AVGTSMR.agg([ lambda x: 'min = ' + str(x.min()), 
                                         lambda x: '50% = ' + str(round(x.median(),2)), 
                                         lambda x: '90% = ' + str(round(x.quantile(0.9),2)), 
                                         lambda x: '99% = ' + str(round(x.quantile(0.99),2)), 
                                         lambda x: '99.9% = ' + str(round(x.quantile(0.99),2))]).reset_index()
print(stat.to_csv(sep='\t', header = None, index = 0))

CANCEL	min = 170	50% = 178.0	90% = 204.4	99% = 222.4	99.9% = 222.4
ORDER	min = 169	50% = 190.0	90% = 249.0	99% = 271.0	99.9% = 271.0
PAYMENT	min = 185	50% = 200.0	90% = 257.0	99% = 276.28	99.9% = 276.28



## Часть 2.

Таблица:
- EXECTIME – Время ответа на транзакцию, кратное 5 микросекундам
- TRANSNO – Количество транзакций с таким временем
- Weight – Процент от общего количества транзакций
- Percent – Процент от общего количества транзакций, имеющих время ответа <= EXECTIME

In [156]:
from tabulate import tabulate
def table(x):
    x = np.ceil(x/5)*5
    ExecTime, TransNo = np.unique(x, return_counts=True)
    Weight = TransNo*100/x.size
    Percent = np.cumsum(Weight)
    result = pd.DataFrame({'ExecTime': ExecTime, 'TransNo': TransNo, 'Weight%': Weight, 'Percent': Percent})
    return(result)

keys = df.EVENT.unique()
for key in keys:
    temp_group = df.groupby('EVENT').get_group(key)
    res = table(temp_group['AVGTSMR'].values)
    print(key)
    print(tabulate(res, headers='keys', tablefmt='fancy_grid', showindex=False))

ORDER
╒════════════╤═══════════╤══════════════╤══════════════╕
│   ExecTime │   TransNo │      Weight% │      Percent │
╞════════════╪═══════════╪══════════════╪══════════════╡
│        170 │        81 │  0.00810183  │   0.00810183 │
├────────────┼───────────┼──────────────┼──────────────┤
│        175 │      5954 │  0.595535    │   0.603636   │
├────────────┼───────────┼──────────────┼──────────────┤
│        180 │     80566 │  8.05842     │   8.66206    │
├────────────┼───────────┼──────────────┼──────────────┤
│        185 │    195142 │ 19.5186      │  28.1807     │
├────────────┼───────────┼──────────────┼──────────────┤
│        190 │    224264 │ 22.4315      │  50.6121     │
├────────────┼───────────┼──────────────┼──────────────┤
│        195 │    169721 │ 16.9759      │  67.5881     │
├────────────┼───────────┼──────────────┼──────────────┤
│        200 │     94244 │  9.42653     │  77.0146     │
├────────────┼───────────┼──────────────┼──────────────┤
│        205 │     39717 